### **Preprocessing**

In [2]:
import re

# Load the file content
file_path = 'nfr.txt'
with open(file_path, 'r') as file:
    content = file.read()

# Remove non-ASCII characters
cleaned_content = re.sub(r'[^\x00-\x7F]', '', content)

# save the cleaned content to a new file
with open('cleaned_nfr.txt', 'w') as file:
    file.write(cleaned_content)


###**Categorization**
for creating custom entities file.

In [4]:
from collections import defaultdict
requirements = defaultdict(list)


for line in cleaned_content.splitlines():
    if line.strip():
        parts = line.split(":", 1)
        if len(parts) == 2:
            prefix = parts[0].strip()
            content = parts[1].strip()

            # Add the content to the appropriate category in the dictionary
            requirements[prefix].append(content)

# number of requirements in each category
for key, reqs in requirements.items():
    print(f"{key}: {len(reqs)} requirements")

PE: 54 requirements
LF: 38 requirements
US: 67 requirements
A: 21 requirements
SE: 66 requirements
F: 255 requirements
L: 13 requirements
O: 62 requirements
SC: 21 requirements
FT: 10 requirements
MN: 17 requirements
PO: 1 requirements


In [5]:
category_key = "PO"  # Replace with required key
if category_key in requirements:
    category_requirements = requirements[category_key]

    print(f"Requirements for category '{category_key}':")
    for req in category_requirements:
        print(f"- {req}")

Requirements for category 'PO':
- The product is expected to run on Windows CE and Palm operating systems.


###**Text Extraction using Custom NER**

In [7]:
#pip install spacy

In [9]:
#!python -m spacy download en_core_web_sm

In [12]:
import spacy
import json
from spacy.matcher import PhraseMatcher
nlp = spacy.load("en_core_web_sm")

# Define custom NER function
def custom_ner(doc, custom_entities):
    actors = []
    actions = []
    conditions = []

    # Creating phrase matchers for multi-token entities
    matcher = PhraseMatcher(nlp.vocab)

    # Ading custom entities to matchers
    actor_patterns = [nlp.make_doc(actor.lower()) for actor in custom_entities['Actor']]
    action_patterns = [nlp.make_doc(action.lower()) for action in custom_entities['Action']]
    condition_patterns = [nlp.make_doc(condition.lower()) for condition in custom_entities['Condition']]

    matcher.add("ACTOR", None, *actor_patterns)
    matcher.add("ACTION", None, *action_patterns)
    matcher.add("CONDITION", None, *condition_patterns)

    # Finding matches for actors, actions, and conditions
    matches = matcher(doc)

    for match_id, start, end in matches:
        span = doc[start:end].text
        label = nlp.vocab.strings[match_id]

        if label == "ACTOR":
            actors.append(span)
        elif label == "ACTION":
            actions.append(span)
        elif label == "CONDITION":
            conditions.append(span)

    return actors, actions, conditions, []

# Function to process requirements
def process_requirements(requirements, custom_entities, replace_terms=None):
    results = []
    for requirement in requirements:
        if replace_terms:
            for old_term, new_term in replace_terms.items():
                requirement = requirement.replace(old_term, new_term)

        # Tokenize and analyze the text with spaCy
        doc = nlp(requirement)

        # Extract custom entities
        actors, actions, conditions, _ = custom_ner(doc, custom_entities)

        # For now, use the entire requirement as the outcome
        outcomes = [requirement]

        results.append({
            "Requirement": requirement,
            "Actors": actors,
            "Actions": actions,
            "Conditions": conditions,
            "Outcomes": outcomes
        })
    return results

# Loading custom entities from JSON file and convert to dictionary
with open('/content/custom_entities.json', 'r') as f:
    custom_entities = json.load(f)

#loading requiremnts
requirements_po = requirements['PO']
requirements_ft = requirements['FT']
requirements_l = requirements['L']
requirements_mn = requirements['MN']
requirements_a = requirements['A']
requirements_sc = requirements['SC']
requirements_lf = requirements['LF']
requirements_pe = requirements['PE']
requirements_se = requirements['SE']
requirements_o = requirements['O']
requirements_us = requirements['US']
requirements_f = requirements['F']

# Process each set of requirements
results_po = process_requirements(requirements_po, custom_entities['PO'])
results_ft = process_requirements(requirements_ft, custom_entities['FT'], replace_terms={"shall": "should"})
results_l = process_requirements(requirements_l, custom_entities['L'], replace_terms={"shall": "should", "will": "should", "must": "should"})
results_mn = process_requirements(requirements_mn, custom_entities['MN'], replace_terms={"shall": "should", "will": "should", "must": "should"})
results_a = process_requirements(requirements_a, custom_entities['A'], replace_terms={"shall": "should", "will": "should", "must": "should"})
results_sc = process_requirements(requirements_sc, custom_entities['SC'], replace_terms={"shall": "should", "will": "should", "must": "should"})
results_lf = process_requirements(requirements_lf, custom_entities['LF'], replace_terms={"shall": "should", "will": "should", "must": "should"})
results_pe = process_requirements(requirements_pe, custom_entities['PE'], replace_terms={"shall": "should", "will": "should", "must": "should"})
results_se = process_requirements(requirements_se, custom_entities['SE'], replace_terms={"shall": "should", "will": "should", "must": "should"})
results_o = process_requirements(requirements_o, custom_entities['O'], replace_terms={"shall": "should", "will": "should", "must": "should"})
results_us = process_requirements(requirements_us, custom_entities['US'], replace_terms={"shall": "should", "will": "should", "must": "should"})
results_f = process_requirements(requirements_f, custom_entities['F'], replace_terms={"shall": "should", "will": "should", "must": "should"})

# Combine all results
output = {
    "PO": results_po,
    "FT": results_ft,
    "L": results_l,
    "MN": results_mn,
    "A": results_a,
    "SC": results_sc,
    "LF": results_lf,
    "PE": results_pe,
    "SE": results_se,
    "O": results_o,
    "US": results_us,
    "F": results_f
}

# Save results to a JSON file
with open('parsed_requirements.json', 'w') as f:
    json.dump(output, f, indent=2)

print("Parsed data has been saved to 'parsed_requirements.json'")


Parsed data has been saved to 'parsed_requirements.json'


###

```
# This is formatted as code
```

**Test case Generation using Rule-based approach**

In [20]:
import json

def parse_requirements(requirements_data):
    if isinstance(requirements_data, str):
        try:
            return json.loads(requirements_data)
        except json.JSONDecodeError:
            print(f"Error decoding JSON: {requirements_data}")
            return {}
    return requirements_data

def rule_based_test_case_generator(parsed_requirements):
    test_cases = []

    for category, requirements in parsed_requirements.items():
        for req in requirements:
            requirement = parse_requirements(req)

            if isinstance(requirement, dict):
                actions = requirement.get('Actions', [])
                actors = requirement.get('Actors', [])
                conditions = requirement.get('Conditions', ['No specific conditions identified'])
                outcomes = requirement.get('Outcomes', [])

                # Generate multiple test cases by combining actions, actors, and conditions
                for action in actions:
                    for actor in actors:
                        for condition in conditions:
                            for outcome in outcomes:
                                test_case = {
                                    'Test Case Title': generate_test_case_title(requirement, action, condition),
                                    'Test Steps': generate_test_steps(requirement, action, condition),
                                    'Preconditions': generate_preconditions(requirement, condition),
                                    'Expected Result': generate_expected_result(requirement)
                                }
                                test_cases.append(test_case)
            else:
                print(f"Skipping invalid requirement format: {requirement}")

    return test_cases

def generate_test_case_title(requirement, action, condition):
    actor = requirement.get('Actors', [''])[0]
    title = f"Verify {action} by {actor}"
    if condition and condition != 'No specific conditions identified':
        title += f" with {condition}"
    return title

def generate_test_steps(requirement, action, condition):
    steps = []
    steps.append(f"Step 1: Ensure the following conditions are met: {condition}.")
    steps.append(f"Step 2: Perform the action '{action}'.")
    steps.append(f"Step 3: Verify the outcome matches the expected result: '{requirement.get('Outcomes', [''])[0]}'")
    return '\n'.join(steps)

def generate_preconditions(requirement, condition):
    return condition

def generate_expected_result(requirement):
    return requirement.get('Outcomes', ['No expected result provided'])[0]


with open('/content/parsed_requirements.json') as f:
    parsed_requirements = json.load(f)

# Generate test cases using rule-based approach
test_cases = rule_based_test_case_generator(parsed_requirements)


def print_first_50_test_cases(test_cases):

    print("The first 50 Test cases\n")

    for i, case in enumerate(test_cases[:50], 1):  # Print only the first 100 test cases
        print(f"Test Case {i}")
        print("-" * 80)
        print(f"Title: {case['Test Case Title']}")
        print("Steps:")
        print(f"{case['Test Steps']}")
        print(f"Preconditions: {case['Preconditions']}")
        print(f"Expected Result: {case['Expected Result']}")
        print("\n" + "=" * 80 + "\n")



print_first_50_test_cases(test_cases)




The first 50 Test cases

Test Case 1
--------------------------------------------------------------------------------
Title: Verify create by product with exception log
Steps:
Step 1: Ensure the following conditions are met: exception log.
Step 2: Perform the action 'create'.
Step 3: Verify the outcome matches the expected result: 'The product should create an exception log of problems encountered within the product for transmission to our company for analysis and resolution.'
Preconditions: exception log
Expected Result: The product should create an exception log of problems encountered within the product for transmission to our company for analysis and resolution.


Test Case 2
--------------------------------------------------------------------------------
Title: Verify create by product with exception log
Steps:
Step 1: Ensure the following conditions are met: exception log.
Step 2: Perform the action 'create'.
Step 3: Verify the outcome matches the expected result: 'The product sh

###**Evaluation**

In [21]:
from sklearn.metrics import precision_score, recall_score, f1_score

# Assume binary labels for match (1) and mismatch (0) for each attribute
y_true = [1, 1, 0, 1]  # True labels
y_pred = [1, 0, 0, 1]  # Predicted labels

precision = precision_score(y_true, y_pred)
recall = recall_score(y_true, y_pred)
f1 = f1_score(y_true, y_pred)

print(f"Precision: {precision:.2f}")
print(f"Recall: {recall:.2f}")
print(f"F1 Score: {f1:.2f}")

Precision: 1.00
Recall: 0.67
F1 Score: 0.80


In [22]:
from sklearn.metrics import accuracy_score

accuracy = accuracy_score(y_true, y_pred)
print(f"Accuracy: {accuracy:.2f}")



Accuracy: 0.75
